In [ ]:
from utils import DatasetBuilder
import yaml
from pyteomics import mgf
from rdkit import Chem
import pubchempy as pcp
from pyteomics import mgf
from dataset import load_cand_data_mzi
from utils import DatasetBuilder, collate_spectra_data, Spectra_data
import torch
import numpy as np
from train_contr import train_contr
from torch.utils.data import DataLoader

ModuleNotFoundError: No module named 'dgl'

In [26]:
def init_datadir(dataset_builder):
# Initialize dataset_builder object with blank values because we will not need it
    dataset_builder.data_dict = []
    dataset_builder.mol_dict = []
    dataset_builder.pair_dict = []
    dataset_builder.split_dict = []
    dataset_builder.fp_dict = []
    dataset_builder.in_to_id_dict = []
    dataset_builder.in_to_id_dict_wneg = []
    dataset_builder.data_dir = './data/NPLIB1/'

def get_from_pubchem(in_smiles):
    m = Chem.MolFromSmiles(in_smiles)
    form = Chem.rdMolDescriptors.CalcMolFormula(m)
    smiles = []
    try:
        compounds = pcp.get_compounds(form, 'formula',record_format='json')
    except:
        return smiles
    for compound in compounds:
        smiles.append(compound.canonical_smiles)
    return smiles
    
def norm_mzi(mz, inten): #normalize intensoty to 999 because that is how JESTR has been trained. Drop mz < 1000 Da
    mz = mz.reshape(-1, 1)
    inten = inten/max(inten)
    inten = inten * 999
    inten = inten.reshape(-1, 1)
    mzi = np.hstack((inten, mz))
    idx_mz = mzi[:,1] <= 1000
    mzi = mzi[idx_mz]
    return mzi

In [33]:
with open('params.yaml') as f:
    params = yaml.load(f, Loader=yaml.FullLoader)
    
import torch

dir_path = ""
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ms_intensity_threshold = 0.0

dataset_builder = DatasetBuilder(params['exp'])
init_datadir(dataset_builder)
molgraph_dict = {}
data_path = dir_path + dataset_builder.data_dir

dataset_builder.molgraph_dict = molgraph_dict
logfile = './trial_notebook.log'
output = open(logfile, 'a')

NameError: name 'DatasetBuilder' is not defined

In [ ]:
# Assign some environment variables, either new or override the ones from params.yaml file
load_from_pubchem = True #Try and download candidates from pubchem if set True, else assume a list is given
# spec_id_from_mgf = 'CCMSLIB00000855758' #The two in sample mgf file are CCMSLIB00000855758, CCMSLIB00000846187
# #The matching pairs are CCMSLIB00000855758 and CC1=CC2C=C(C)C(C)C3C(CC(C)C)NC(=O)C23C(=O)CCC2OC(C)(C)OC2CC1
# #and CCMSLIB00000846187 and COc1c2c(c(C)c3c1C(=O)OC3)OC1(C)CC3(O)OCC4(C)OC5(OC)CCC(C)(C1C2)C43O5
# target_smiles = 'CC1=CC2C=C(C)C(C)C3C(CC(C)C)NC(=O)C23C(=O)CCC2OC(C)(C)OC2CC1'
# mgf_file = data_path + 'my_data.mgf' #path of mgf file

In [ ]:
# List of candidate SMILES if user doesnt want to download from PubChem. 
# Note that the molecules in PubChem keep changing. For the lists given below, JESTR gives a rank of 1 for
# both examples. This rank may change on any given day because the list of molecules retrieved from PubChem
# could change, and harder molecules may appear in the retrieved list

candidate_dict = {}

The below code was not actually run by me, but it shows what I was attempting to do with JESTR. 

In [ ]:
target_smiles_list = ["CN1CCC(CC1)NC2=NC3=C(C(=O)NC=C3Br)C(=N2)NC4=CC=C(C=C4)OC5=CC=CC=C5", "C1=CC2=C(C=C1O)C(=CN2)CCO", "CN1C=C2C=CC=C(C2=N1)CN3C[C@H]4N([C@H](C3=O)CC5=CC=C(C=C5)OP(=O)(O)O)C(=O)CN(N4C(=O)NCC6=CC=CC=C6)CC=C", "CCCCCCCCCCCCCC(=O)OC(CC(=O)[O-])C[N+](C)(C)C", "CNC1=NC(=C(S1)NC(=O)C2=CC(=C(C=C2)OC)OC)C(=O)N", "CC1=C(C(C2=C(N1)CC(CC2=O)(C)C)C3=CC(=C(C(=C3)Br)O)OC)C(=O)OC4CCCC4", "CCCCCCC(CCCCCCCCCCC(=O)O)OC(=O)CCCCCCCCCCC=C", "CC1(CCC2(CCC3(C(=CCC4C3(CCC5C4(CCC(C5(C)CO)O)C)C)C2C1)C)C(=O)O)C", "COC1=C(C=C(C=C1)C2CC(=O)C3=C(O2)C=CC(=C3)Cl)OC", "C1=C(N=C(C(=O)N1)C(=O)N)F"]

#Instantiate models and load weights
params['load_dicts'] = False
params['pretrained_mol_enc_model'] = 'data/NPLIB1/pretrained_mol_enc_model_1707829192911_best.pt'
params['pretrained_spec_enc_model'] = 'data/NPLIB1/pretrained_spec_enc_model_1707829192911_best.pt'

mol_enc_model_contr, spec_enc_model_contr, models_list = train_contr(dataset_builder, molgraph_dict, params, output,
                                                             device, data_path, True)
mol_enc_model_contr.eval()
spec_enc_model_contr.eval()
i = 0

for smiles in target_smiles_list:

        spec_id_from_mgf = str(i) #The two in sample mgf file are CCMSLIB00000855758, CCMSLIB00000846187
        #The matching pairs are CCMSLIB00000855758 and CC1=CC2C=C(C)C(C)C3C(CC(C)C)NC(=O)C23C(=O)CCC2OC(C)(C)OC2CC1
        #and CCMSLIB00000846187 and COc1c2c(c(C)c3c1C(=O)OC3)OC1(C)CC3(O)OCC4(C)OC5(OC)CCC(C)(C1C2)C43O5
        target_smiles = smiles
        mgf_file = data_path + 'test_new_ten.mgf' #path of mgf file

        if smiles not in candidate_dict.keys():
                candidate_smiles = get_from_pubchem(smiles)
                assert len(candidate_smiles) != 0, "No candidates found!"
        else:
                candidate_smiles = candidate_dict[target_smiles]

        try:
                spec = mgf.get_spectrum(mgf_file, spec_id_from_mgf)
        except:
                assert False, "No spectrum found in MGF file"
                
        assert spec['params']['charge'][0] == 1, "Not an [M+H]+ spectrum"

        mz = spec['m/z array']
        intensity = spec['intensity array']
        mzi = norm_mzi(mz, intensity)

        data_list = load_cand_data_mzi(dataset_builder, params, target_smiles, candidate_smiles, mzi, device)

        assert len(data_list) >= 2, "Candidates could not be converted to rdkit mol objects"    
        spec_test_ds = Spectra_data(data_list)
        collate_fn = collate_spectra_data(molgraph_dict, params)

        dl_params = {'batch_size': params['batch_size_val_final'],
                'shuffle': False}
        spec_test_dl = DataLoader(spec_test_ds, collate_fn=collate_fn, **dl_params)
        mol_enc_total = torch.Tensor()
        spec_enc_total = torch.Tensor()

        for batch_id, (batch_g, mz_b, int_b, pad, fp_b, y, lengths, inchi) in enumerate(spec_test_dl):
                batch_g = batch_g.to(torch.device(device))
                mz_b = mz_b.to(torch.device(device))
                int_b = int_b.to(torch.device(device))
                pad = pad.to(torch.device(device))
                fp_b = fp_b.to(torch.device(device))
                y = y.to(torch.device(device))
                with torch.no_grad():
                        mol_enc = mol_enc_model_contr(batch_g, batch_g.ndata['h'])
                        spec_enc = spec_enc_model_contr(mz_b, int_b, pad, lengths)

        mol_enc_total = torch.cat([mol_enc_total, mol_enc.cpu()])
        spec_enc_total = torch.cat([spec_enc_total, spec_enc.cpu()])

        dist = torch.nn.CosineSimilarity()
        dist = dist(mol_enc_total, spec_enc_total)
        dist = dist.tolist()
        dist = np.array(dist)
        rank_dist = (dist[0] > dist[1:]).sum()
        rank_dist = len(dist) - rank_dist

        print("Rank for Target SMILES {} for spectrum ID {} = {}".format(target_smiles, spec_id_from_mgf, rank_dist))
        i+=1

In [ ]:
# #Load spectrum from MGF file
# try:
#     spec = mgf.get_spectrum(mgf_file, spec_id_from_mgf)
# except:
#     assert False, "No spectrum found in MGF file"
    
# assert spec['params']['charge'][0] == 1, "Not an [M+H]+ spectrum"

# mz = spec['m/z array']
# intensity = spec['intensity array']
# mzi = norm_mzi(mz, intensity)

In [ ]:
# #load candidate data and check if rdkit mols can be generated for all candidates
# data_list = load_cand_data_mzi(dataset_builder, params, target_smiles, candidate_smiles, mzi, device)

# assert len(data_list) >= 2, "Candidates could not be converted to rdkit mol objects"

In [ ]:
# #Instantiate models and load weights
# params['load_dicts'] = False
# params['pretrained_mol_enc_model'] = 'data/NPLIB1/pretrained_mol_enc_model_1707829192911_best.pt'
# params['pretrained_spec_enc_model'] = 'data/NPLIB1/pretrained_spec_enc_model_1707829192911_best.pt'

# mol_enc_model_contr, spec_enc_model_contr, models_list = train_contr(dataset_builder, molgraph_dict, params, output,
#                                                              device, data_path, True)
# mol_enc_model_contr.eval()
# spec_enc_model_contr.eval()

11-17 16:45:46	loading pretrained_mol_enc_model weights from data/NPLIB1/pretrained_mol_enc_model_1707829192911_best.pt
11-17 16:45:46	loading pretrained_spec_enc_model weights from data/NPLIB1/pretrained_spec_enc_model_1707829192911_best.pt


SpecEncMLP_BIN(
  (dropout): Dropout(p=0.4, inplace=False)
  (mz_fc1): Linear(in_features=1000, out_features=1024, bias=True)
  (mz_fc2): Linear(in_features=1024, out_features=1024, bias=True)
  (mz_fc3): Linear(in_features=1024, out_features=512, bias=True)
  (relu): ReLU()
)

In [ ]:
# # set up dataloaders
# spec_test_ds = Spectra_data(data_list)
# collate_fn = collate_spectra_data(molgraph_dict, params)

# dl_params = {'batch_size': params['batch_size_val_final'],
#              'shuffle': False}
# spec_test_dl = DataLoader(spec_test_ds, collate_fn=collate_fn, **dl_params)
# mol_enc_total = torch.Tensor()
# spec_enc_total = torch.Tensor()

In [ ]:
# #Run encoders to get embeddings
# for batch_id, (batch_g, mz_b, int_b, pad, fp_b, y, lengths, inchi) in enumerate(spec_test_dl):
#     batch_g = batch_g.to(torch.device(device))
#     mz_b = mz_b.to(torch.device(device))
#     int_b = int_b.to(torch.device(device))
#     pad = pad.to(torch.device(device))
#     fp_b = fp_b.to(torch.device(device))
#     y = y.to(torch.device(device))
#     with torch.no_grad():
#         mol_enc = mol_enc_model_contr(batch_g, batch_g.ndata['h'])
#         spec_enc = spec_enc_model_contr(mz_b, int_b, pad, lengths)

#     mol_enc_total = torch.cat([mol_enc_total, mol_enc.cpu()])
#     spec_enc_total = torch.cat([spec_enc_total, spec_enc.cpu()])

In [ ]:
# #Calculate rank
# dist = torch.nn.CosineSimilarity()
# dist = dist(mol_enc_total, spec_enc_total)
# dist = dist.tolist()
# dist = np.array(dist)
# rank_dist = (dist[0] > dist[1:]).sum()
# rank_dist = len(dist) - rank_dist

In [ ]:
# print("Rank for Target SMILES {} for spectrum ID {} = {}".format(target_smiles, spec_id_from_mgf, rank_dist))

Rank for Target SMILES CC1=CC2C=C(C)C(C)C3C(CC(C)C)NC(=O)C23C(=O)CCC2OC(C)(C)OC2CC1 for spectrum ID CCMSLIB00000855758 = 7
